In [1]:
#%env SM_FRAMEWORK=tf.keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfkl
#constantes
RIR_PATH = '/home/mrtn/Documents/TESIS/de-reverb/bases de datos/data/impulsos'
SPEECH_PATH = '/home/mrtn/Documents/TESIS/de-reverb/bases de datos/data/speech/train-clean-100'
Q_e = 32
LM = 5
Pd = 9

In [39]:
tf.keras.backend.clear_session()

input_A = tfkl.Input((256, LM,1)) #DIMENSIONES K x L x C
input_B = tfkl.Input((256, Pd,1))
delay = tfkl.Reshape((256, 1, Pd))(input_B)

encoder_0 = tfkl.Conv2D(16, kernel_size=(1,LM), strides=(2,1), name='Input_layer')(input_A)
encoder_1 = tfkl.Conv2D(16, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_1')(encoder_0)
encoder_2 = tfkl.Conv2D(32, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_2')(encoder_1)
encoder_3 = tfkl.Conv2D(32, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_3')(encoder_2)
encoder_4 = tfkl.Conv2D(64, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_4')(encoder_3)

encoder_5 = tfkl.Conv2D(64, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_5')(encoder_4)

decoder_6 = tfkl.Conv2DTranspose(64, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_6')(encoder_5)
decoder_6 = tfkl.Add()([encoder_4, decoder_6])
decoder_7 = tfkl.Conv2DTranspose(32, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_7')(decoder_6)
decoder_7 = tfkl.Add()([encoder_3, decoder_7])
decoder_8 = tfkl.Conv2DTranspose(32, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_8')(decoder_7)
decoder_8 = tfkl.Add()([encoder_2, decoder_8])
decoder_9 = tfkl.Conv2DTranspose(16, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_9')(decoder_8)
decoder_9 = tfkl.Add()([encoder_1, decoder_9])
decoder_10 = tfkl.Conv2DTranspose(16, kernel_size=(9,1), strides=(2,1), activation='relu',padding='same',name='hidden_10')(decoder_9)
decoder_10 = tfkl.Add()([encoder_0, decoder_10])
decoder_11 = tfkl.Conv2DTranspose(Pd, kernel_size=(9,1), strides=(2,1), activation='linear',padding='same',name='hidden_11')(decoder_10)

out = tfkl.multiply([delay, decoder_11])
out = tf.reduce_sum(out, 3)
out = tfkl.Activation('relu')(out)

ae = tf.keras.Model(inputs=[input_A, input_B], outputs=[out])

ae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')


In [40]:
ae.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 5, 1)]  0                                            
__________________________________________________________________________________________________
Input_layer (Conv2D)            (None, 128, 1, 16)   96          input_1[0][0]                    
__________________________________________________________________________________________________
hidden_1 (Conv2D)               (None, 64, 1, 16)    2320        Input_layer[0][0]                
__________________________________________________________________________________________________
hidden_2 (Conv2D)               (None, 32, 1, 32)    4640        hidden_1[0][0]                   
______________________________________________________________________________________________

In [63]:
"""Alimentar con datos al bucle de entrenamiento"""
import keras
import numpy as np
import os
import glob
import random
import librosa

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, path='', batch_size=8, dim=(32767), n_channels=1,  shuffle=True):
        'Initialization'
        self.path = path
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        input_A = np.empty((self.batch_size, 257, 5))
        input_B = np.empty((self.batch_size, 257, 9))
        output = np.empty((self.batch_size, 257, 1))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):

            input_A, input_B, output = np.load(self.path + str(ID) + '.npy', allow_pickle = True)

        return [input_A[:-1,:].reshape(1,256,5,1), input_B[:-1,:].reshape(1,256,9,1)], output[:-1].reshape(1,256,1)


def build_generators(params):
    """
    Crea instancias de la clase DataGenerator (para entrenamiento y valdiacion) a partir de un diccionario donde se determinan los parametros
    del generador de datos, y el path principal.
    PARAMETROS:
        -MAIN_PATH (str) path principal de la carpeta de trabajo
        -params (dict) diccionario con los campos 'dim'(int), 'batch_size'(int), 'shuffle'(bool) para configurar el generador de datos
        -subpath (str) path de la carpeta dentro de data/ de donde tomar los datos. Por defecto esta asignada a 'data_ready' que es donde se encuentran
        los datos procesados. Puede ser util cambiarla a la carpeta data_dummy para trabajar con los datos dummy en ocasiones de debuggeo
    SALIDA:
        -training_generator (DataGenerator) instancia de clase que contiene los datos para pasarse a una instancia de entrenamiento y proveer
            los datos de entrenamiento al modelo
        -validation_generator (DataGenerator) instancia de clase que contiene los datos para pasarse a una instancia de entrenamiento y proveer
            los datos de validacion al modelo
        """

    audio_list = glob.glob(params['path']+'/**/*.npy', recursive = True)

    #seleccion random de sets
    audio_numbers = list(range(0, len(audio_list)))
    random.shuffle(audio_numbers)
    train_n = int(len(audio_numbers)*0.9)
    validation_n = len(audio_numbers) - train_n

    partition = {'train' : audio_numbers[:train_n], 'validation' : audio_numbers[train_n:]}

    # Generators
    training_generator = DataGenerator(partition['train'], **params)
    validation_generator = DataGenerator(partition['validation'], **params)

    print('Cantidad de datos para entrenamiento:', len(partition['train']))
    print('Cantidad de datos para validacion:', len(partition['validation']))
    return training_generator, validation_generator


In [64]:
in_a = []
in_b = []
out = []
for i in range(1636):
    input_A, input_B, output = np.load('npy_data/' + str(i) + '.npy', allow_pickle = True)
    in_a.append(input_A[:-1,:].reshape(1,256,5,1))
    in_b.append(input_B[:-1,:].reshape(1,256,9,1))
    out.append(output[:-1].reshape(1,256,1))

In [65]:
#Data generators
params = {'path':'/home/mrtn/Documents/TESIS/de-reverb/nuevo_modelo/npy_data/', 'batch_size' : 16, 'dim' : (256, 256)}
training_generator, validation_generator = build_generators(params)

#cbks = [tf.keras.callbacks.EarlyStopping(monitor='loss',restore_best_weights=True, patience=2),
#        tf.keras.callbacks.ModelCheckpoint('/home/martin/Documents/tesis/src/model/ckpts/weights.{epoch:02d}-{loss:.3f}.hdf5'),
#        tf.keras.callbacks.TensorBoard(log_dir='tb_logs',profile_batch=0, update_freq='batch', histogram_freq=1)]

#Entreno
ae.fit(training_generator, epochs=10)
#ae.fit(x = [in_b, in_a], y = out, batch_size=32, epochs=10)


Cantidad de datos para entrenamiento: 1472
Cantidad de datos para validacion: 164
Epoch 1/10
92/92 [==============================] - 1s 7ms/step - loss: 0.6544
Epoch 2/10
92/92 [==============================] - 1s 7ms/step - loss: 0.4613
Epoch 3/10
92/92 [==============================] - 1s 7ms/step - loss: 0.4884
Epoch 4/10
92/92 [==============================] - 1s 7ms/step - loss: 0.6109
Epoch 5/10
92/92 [==============================] - 1s 7ms/step - loss: 0.6761
Epoch 6/10
92/92 [==============================] - 1s 7ms/step - loss: 0.6607
Epoch 7/10
92/92 [==============================] - 1s 7ms/step - loss: 0.5777
Epoch 8/10
92/92 [==============================] - 1s 7ms/step - loss: 0.5904
Epoch 9/10
92/92 [==============================] - 1s 7ms/step - loss: 0.4610
Epoch 10/10
92/92 [==============================] - 1s 7ms/step - loss: 0.6244


In [36]:
out[0].shape

(1, 256, 1)